<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<h2>Deep Learning en el diagnóstico de Alzheimer mediante Imágenes de Resonancia Magnética</h2> 


Procedimiento de entrenamiento con imágenes en formato cerebrales, extraídas previamente de imágenes en formato .NIFTI a JPG

## Instalar librerías

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install scipy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install kiwisolver

In [ ]:
!pip install -U Pillow

In [ ]:
!pip install pandas

In [ ]:
!pip3 --upgrade pandas

In [6]:
!pip3 install --upgrade --force-reinstall h5py

  Using cached h5py-3.6.0-cp37-cp37m-win_amd64.whl (2.8 MB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'c:\\users\\juan david\\.conda\\envs\\tensorflow-ml\\lib\\site-packages\\~-mpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [34]:
!pip install tensorflow-datasets

  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=299073844efb31e6f136a5dc52b2a277e44fe49492c62a7d0a6eb5bc1cd8465a
  Stored in directory: c:\users\juan david\appdata\local\pip\cache\wheels\29\93\c6\762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
Successfully built promise


## Importar librerías

In [41]:
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow 
from tensorflow import keras
from scipy import ndimage, misc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import time
import os
import pandas as pd
import shutil
from pathlib import Path
import datetime
import cv2
import skimage.exposure as e
from scipy.signal import convolve2d

import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import nibabel as nib
from skimage.io import imread_collection
import time

## Carga y lectura de datos

Se cargarán las imágenes en formato JPG almacenadas en el directorio local
el cual se creó en el Notebook previo 1. Extracción_Imágenes_Nifti_A_Jpg. Dichas imágenes serán almacenadas en los siguientes conjuntos <b>datos entrenamiento, validación y prueba.</b>

Los datos utilizados en este proyecto fueron dispuestos por el archivo de imágenes y datos <a href="https://adni.loni.usc.edu/data-samples/access-data/">(IDA)</a> está a cargo del Laboratorio de neuroimagen (LONI) en el Instituto de informática y neuroimagen Mark and Mary Stevens de la USC. Del cual se descargaron 2294 imágenes en formato .nii de 639 pacientes, con un periodo de 3 años de tomas de IMR para el Alzheimer, en personas entre los 55 y 91 años de edad. Dichas imágenes se convirtieron en imágenes en formato .jpg, duplicando la cantidad de las mismas, ya que por cada imagen 3D se extrajo la dimensión del corte axial y corte coronal, las cuales brindan información válida para la detección de la enfermedad del Alzheimer, ignorando así el corte sagital y obteniendo una totalidad de 4588 imágenes en formato JPG.

In [42]:
"""Categorización datos de entrenamiento, validación y prueba

1. Imágenes disponibles en jpg (Corte Axial y Coronal)

clasificación  cantidad
AD             952
CN             1410
MCI            2226
-----------------------
TOTAL          4588

2. División por cada conjunto de datos

clasificación  cantidad  entrenamiento (80%)  validación (19%)  prueba (1%)
AD             952       762                  181               9
CN             1410      1128                 268               14
MCI            2226      1781                 423               22
---------------------------------------------------------------------------
TOTAL          4588      3671                 872               45

"""

df_conjuntos = pd.DataFrame(columns=['clasificacion', 'cantidad' ,'entrenamiento','validacion', 'prueba'])
df_conjuntos.loc[1] = ['AD', 952, 762, 181, 9]  
df_conjuntos.loc[2] = ['CN', 1410, 1128, 268, 14]
df_conjuntos.loc[3] = ['MCI', 2226, 1781, 423, 22]
df_conjuntos

,clasificacion,cantidad,entrenamiento,validacion,prueba
1,AD,952,762,181,9
2,CN,1410,1128,268,14
3,MCI,2226,1781,423,22


In [43]:
%run ./Helpers/sqllite_helper.ipynb
helper_sqllite()

helper sqllite loaded..


In [44]:
database = r"C:\sqllite\alzheimer_db.db"
conn = create_connection(database)

query = "SELECT * FROM log_metadatos_consolidado"
df_adni = query_table(conn, query)
df_adni

,index,image_data_id,subject,path_nii,file_name_nii,path_jpg,file_name_jpg,state,msg,creation_date,...,sex,age,visit,modality,description,type,acq date,format,downloaded,orden
0,0,I45108,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,85,1,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,4/18/2006,NiFTI,None,0
1,1,I45108,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,85,1,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,4/18/2006,NiFTI,None,1
2,2,I40966,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,85,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,11/02/2006,NiFTI,None,2
3,3,I40966,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,85,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,11/02/2006,NiFTI,None,3
4,4,I64025,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,86,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,5/25/2007,NiFTI,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4583,4583,I97341,941_S_1311,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR-R__GradWarp__B1_Correct...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR-R__GradWarp__B1_Correct...,Exito,Exito,2022-06-19 12:10:40,...,M,70,3,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,9/27/2007,NiFTI,None,4583
4584,4584,I97327,941_S_1311,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,69,1,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/02/2007,NiFTI,None,4584
4585,4585,I97327,941_S_1311,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,69,1,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/02/2007,NiFTI,None,4585
4586,4586,I112538,941_S_1311,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\941...,ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,...,M,70,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,6/01/2008,NiFTI,None,4586


In [38]:
df = df_adni[df_adni.state == 'Error']
df['path_jpg'].iloc[0]
df['file_name_nii'].iloc[0]


#ADNI_002_S_0954_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001120143792_S19979_I118688_noneck.nii.gz

#'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI\\002_S_0954\\MPR-R__GradWarp__B1_Correction__N3__Scaled_2\\2006-10-10_15_13_04.0\\I118688\\jpg_no_neck'
#'ADNI_002_S_0954_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001120143792_S19979_I118688_noneck.ni_coronal'

'ADNI_002_S_0954_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001120143792_S19979_I118688_noneck.nii.gz'

<b>Asignación de grupos de datos</b>

Clasificación de conjuntos de datos Entrenamiento, validación y prueba. Adicional se crean 3 columnas calculadas para almacenar los valores de los directorios donde se clasificarán las imágenes en los subconjuntos de datos mencionados previamente con el fin de preprocesar las imágenes y por último utilizarlas para construir el modelo de detección de la enfermedad de Alzheimer, mediante varias arquitecturas de redes neuronales CNN.

In [45]:
result_dataframe = pd.DataFrame()
df_conj_iter = df_conjuntos.iterrows()
for index, row in df_conj_iter:
    print(row['clasificacion'], 
          row['entrenamiento'],
          row['validacion'], 
          row['prueba'])
    
    df_actual = df_adni[df_adni['group_m']==row['clasificacion']]
    df_actual = df_actual.reset_index(level=0)
    df_actual['index1'] = df_actual.index
    
    min_idx_entrenaiento = min(df_actual['index1']) 
    max_idx_entrenamiento = min_idx_entrenaiento + row['entrenamiento']

    min_idx_validacion = max_idx_entrenamiento 
    max_idx_validacion = min_idx_validacion + row['validacion']

    min_idx_prueba = max_idx_validacion
    max_idx_prueba = min_idx_prueba + row['prueba']
    
    tipo_conjunto_1 = 'ENTRENAMIENTO'
    tipo_conjunto_2 = 'VALIDACION'
    tipo_conjunto_3 = 'PRUEBA'    
    
    
    print('min_idx_entrenaiento', min_idx_entrenaiento)
    print('max_idx_entrenamiento', max_idx_entrenamiento, max_idx_entrenamiento - min_idx_entrenaiento)
    
    print('min_idx_validacion', min_idx_validacion)
    print('max_idx_validacion', max_idx_validacion, max_idx_validacion - min_idx_validacion)
    
    print('min_idx_prueba', min_idx_prueba)
    print('max_idx_prueba', max_idx_prueba, max_idx_prueba - min_idx_prueba)
    print('-'*30)
    
    cond_idx_entrenamiento = ((df_actual.index1 >=  min_idx_entrenaiento) & (df_actual.index1 <  max_idx_entrenamiento))
    cond_idx_validacion = ((df_actual.index1 >=  min_idx_validacion) & (df_actual.index1 < max_idx_validacion))
    cond_idx_prueba = ((df_actual.index1 >=  min_idx_prueba) & (df_actual.index1 <=  max_idx_prueba))

    # agregar tipo de conjunto para las imagenes.
    df_actual['tipo_conjunto'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
       tipo_conjunto_1,
       tipo_conjunto_2,
       tipo_conjunto_3
    ], 
    default='_sin_conjunto' )
    
    # agregar ruta de almacenamiento para conjunto de datos de entrenamiento, validación y pruebas
    root_path_conjunto_pre_proc = 'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO'
    df_actual['path_conjunto_pre_procesamiento'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_1 + '\\PROCESADO\\',
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_2 + '\\PROCESADO\\',
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_3 + '\\PROCESADO\\'
    ], 
    default='_sin_conjunto')
    
    # agregar ruta de almacenamiento para conjunto de datos de entrenamiento, validación y pruebas
    root_path_conjunto_deteccion = 'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO'
    df_actual['path_conjunto_deteccion'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_1 + '\\' + row['clasificacion'] + '\\',
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_2 + '\\' + row['clasificacion'] + '\\',
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_3 + '\\' + row['clasificacion'] + '\\'        
    ], 
    default='_sin_conjunto')
    
    # df_actual =  df_actual.rename(columns={"image_data_id": "image_data_id_x", "subject": "subject_x"})   
    result_dataframe = result_dataframe.append(df_actual, ignore_index=True)

result_dataframe.drop(['index', 'level_0', 'index1'], axis = 1, inplace = True) 
print(result_dataframe.shape)

AD 762 181 9
min_idx_entrenaiento 0
max_idx_entrenamiento 762 762
min_idx_validacion 762
max_idx_validacion 943 181
min_idx_prueba 943
max_idx_prueba 952 9
------------------------------
CN 1128 268 14
min_idx_entrenaiento 0
max_idx_entrenamiento 1128 1128
min_idx_validacion 1128
max_idx_validacion 1396 268
min_idx_prueba 1396
max_idx_prueba 1410 14
------------------------------
MCI 1781 423 22
min_idx_entrenaiento 0
max_idx_entrenamiento 1781 1781
min_idx_validacion 1781
max_idx_validacion 2204 423
min_idx_prueba 2204
max_idx_prueba 2226 22
------------------------------
(4588, 23)


In [46]:
result_dataframe.sort_values(by=["orden"], ascending=False, inplace = True)
result_dataframe.tail(3)

,image_data_id,subject,path_nii,file_name_nii,path_jpg,file_name_jpg,state,msg,creation_date,group_m,...,modality,description,type,acq date,format,downloaded,orden,tipo_conjunto,path_conjunto_pre_procesamiento,path_conjunto_deteccion
954,I40966,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,CN,...,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,11/02/2006,NiFTI,None,2,ENTRENAMIENTO,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...
953,I45108,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,CN,...,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,4/18/2006,NiFTI,None,1,ENTRENAMIENTO,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...
952,I45108,002_S_0295,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI\002...,ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correctio...,Exito,Exito,2022-06-19 12:10:40,CN,...,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,4/18/2006,NiFTI,None,0,ENTRENAMIENTO,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...,D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE...


In [47]:
df_tipo_agg = result_dataframe.groupby(["tipo_conjunto"], as_index=False)["path_jpg"].count()
df_tipo_agg

,tipo_conjunto,path_jpg
0,ENTRENAMIENTO,3671
1,PRUEBA,45
2,VALIDACION,872


## Carga datos de pre procesamiento

In [49]:
currentime_ini = datetime.datetime.now()
time = currentime_ini.strftime("%Y-%m-%d %H:%M:%S")
print("Procesando imagenes copiado..", time)

for index, row in result_dataframe.iterrows():
    
    original      = row['path_jpg']  + '\\' + row['file_name_jpg'] + '.jpg'
    target_path   = row['path_conjunto_deteccion'] 
    target        = row['path_conjunto_deteccion'] + row['file_name_jpg'] + '.jpg'
    
    try:
        if not os.path.exists(target_path):
            os.makedirs(target_path)
    
        os.chmod(target_path, 0o777)
        shutil.copyfile(original, target)
    except Exception as e:
        print("Error copiando: ", original)
        print("ERROR : "+str(e))

currentime_fin = datetime.datetime.now()
time = currentime_fin.strftime("%Y-%m-%d %H:%M:%S")
print("Estampa tiempo fin copiado..", time)

diff = currentime_fin - currentime_ini
print("Tiempo total HH:MM:SS.mssss: ",  diff)

Procesando imagenes copiado.. 2022-06-19 20:30:40
Estampa tiempo fin copiado.. 2022-06-19 20:32:14
Tiempo total HH:MM:SS.mssss:  0:01:33.891290


## Balanceo de clases 

### Estrategia: Oversampling de las clases minoritaria

<table style="width:100%" align="center" border="1" >  
    <tr>
      <th>Clasificacion</th>
      <th>Cantidad</th>
      <th>Entrenamiento</th>
      <th>Validacion</th>
      <th>Prueba</th>
    </tr>  
     <tr>
       <td>AD</td>
       <td>952</td>
       <td>762</td>
       <td>181</td>
       <td>9</td>
     </tr>
     <tr>
       <td>CN</td>
       <td>1410</td>
       <td>1128</td>
       <td>268</td>
       <td>14</td>
     </tr>
     <tr>
       <td>MCI</td>
       <td>2226</td>
       <td>1781</td>
       <td>423</td>
       <td>22</td>
     </tr>    
</table>

<p>
Para afrontar uno de los problemas de sobre entrenamiento se procede con la operacion de aumento de datos de manera aleatoria, las imágenes se generarán a partir de varias transformaciones a partir de las originales, con funciones para voltear las imágenes de manera horizontal o vertical, rotación, cambio de color posición de objetos, etc.
</p>

In [50]:
"""Esta función mostrara imágenes en un grid con 1 fila y 10 columnas

Parameters
----------
list : numpy.ndarray
"""

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 1, figsize=(20,20))
    axes =  axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [62]:
# ------------------------------------------------
# Clasificacion	Entrenamiento	Validacion	Prueba
#     AD		     762	        181	      9
#     CN		     1128	        268	      14
#     MCI		     1781	        423	      22
# ------------------------------------------------
# CLASE (CN)
# ENTRENAMIENTO = 1781 - 1128 = 653
# VALIDACION    = 423  - 268  = 155
# PRUEBA        = 22   - 14   = 8
# ------------------------------------------------
# CLASE (AD)
# ENTRENAMIENTO = 1781 - 762  = 1019
# VALIDACION    = 423  - 181  = 242
# PRUEBA        = 22   - 9    = 13
# ------------------------------------------------

datos_faltantes = [
    {
        'clase': 'AD',
        'tipo_conjunto': 'ENTRENAMIENTO',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\\AD\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\ENTRENAMIENTO\\AD\\GENERADOS\\',
        'datos_a_generar': 1019,
        'datos_actuales': 762,
        'datos_total': 1781
    },
    {
        'clase': 'AD',
        'tipo_conjunto': 'VALIDACION',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\AD\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\AD\\GENERADOS\\',
        'datos_a_generar': 242,
        'datos_actuales': 181,
        'datos_total': 423
    },
    {
        'clase': 'AD',
        'tipo_conjunto': 'PRUEBA',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\PRUEBA\\AD\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\PRUEBA\AD\\GENERADOS\\',
        'datos_a_generar': 13,
        'datos_actuales': 9,
        'datos_total': 22
    },
    {
        'clase': 'CN',
        'tipo_conjunto': 'ENTRENAMIENTO',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\ENTRENAMIENTO\\CN\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\ENTRENAMIENTO\\CN\\GENERADOS\\',
        'datos_a_generar': 653,
        'datos_actuales': 1128,
        'datos_total': 1781
    },
    {
        'clase': 'CN',
        'tipo_conjunto': 'VALIDACION',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\CN\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\CN\\GENERADOS\\',
        'datos_a_generar': 155,
        'datos_actuales': 268,
        'datos_total': 423
    },
    {
        'clase': 'CN',
        'tipo_conjunto': 'PRUEBA',
        'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\PRUEBA\\CN\\',
        'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\PRUEBA\CN\\GENERADOS\\',
        'datos_a_generar': 8,
        'datos_actuales': 14,
        'datos_total': 22
    }    
]

datos_faltantes

[{'clase': 'AD',
  'tipo_conjunto': 'ENTRENAMIENTO',
  'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\ENTRENAMIENTO\\AD\\',
  'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\ENTRENAMIENTO\\AD\\GENERADOS\\',
  'datos_a_generar': 1019,
  'datos_actuales': 762,
  'datos_total': 1781},
 {'clase': 'AD',
  'tipo_conjunto': 'VALIDACION',
  'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\AD\\',
  'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\VALIDACION\\AD\\GENERADOS\\',
  'datos_a_generar': 242,
  'datos_actuales': 181,
  'datos_total': 423},
 {'clase': 'AD',
  'tipo_conjunto': 'PRUEBA',
  'path_original': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\PRUEBA\\AD\\',
  'path_generados': 'D:\\ADNI - IDA\\ADNI1_Complete 1Yr 1.5T\\ADNI-PRE-PROCESAMIENTO\\PRUEBA\\AD\\GENERADOS\\',
  'datos_a_generar': 13,
  'datos_actuales'

In [60]:
import time
print_sample = True

count_imagenes_generadas = 5
gen = ImageDataGenerator(rotation_range=10,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.15,
                         zoom_range=0.1,
                         channel_shift_range=10.,
                         horizontal_flip=True)

    
for datos in datos_faltantes:
    dir_data_set = datos['path_original']
    dir_generados = datos['path_generados']
    
    # asignar limite
    rang = datos['datos_a_generar']   
    print(rang)
    print('-'*30)
    
    os.chmod(dir_data_set, 0o777)
    if not os.path.exists(dir_generados):
        os.makedirs(dir_generados, 0o777)       
         
    for idx in range(1, rang + 1):    
        
        chosen_image = random.choice(os.listdir(dir_data_set))
        base = chosen_image.split('.')
        file_name = base[0]
        image_path = dir_data_set + chosen_image
        os.chmod(image_path, 0o777) 
        print(idx)

        # validar la ruta
        print(image_path)
        try:        
            assert os.path.isfile(image_path)
            
             # obtener la imagen
            image = np.expand_dims(plt.imread(image_path), 0)

            image_to_augment = image[0]/255
            image_to_augment = image[0].reshape(1, *image_to_augment.shape,1)
            aug_iter = gen.flow(image_to_augment)        

            current_dir = dir_generados + file_name + '_imagen_generada_' + str(idx) + '.jpg' 
            cv2.imwrite(current_dir, next(aug_iter)[0])

            #time.sleep(2)
        except Exception as e:
            print(image_path)  
            print(str(e))
       

print('fin de imagenes generadas..')            

1019
------------------------------
1
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_023_S_0139_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071113173357620_S17401_I81883_noneck_coronal.jpg
2
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_057_S_0474_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20061229153747701_S13990_I34720_noneck_axial.jpg
3
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_014_S_1095_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080221164939884_S45077_I91459_noneck_coronal.jpg
4
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_012_S_0720_MR_MPR____N3__Scaled_Br_20080225174150688_S17682_I92529_noneck_coronal.jpg
5
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_023_S_0083_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001153941224_S10568_I118874_noneck_coronal.jpg
6
D:\ADNI - ID

46
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_123_S_0088_MR_MPR-R____N3__Scaled_2_Br_20081006154639276_S17840_I119283_noneck_axial.jpg
47
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_010_S_0829_MR_MPR____N3__Scaled_Br_20070731163202385_S26117_I63349_noneck_axial.jpg
48
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_029_S_0836_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070718142657136_S27848_I60692_noneck_coronal.jpg
49
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_027_S_1385_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070330094058546_S28133_I47574_noneck_axial.jpg
50
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_007_S_1339_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070607134807952_S27414_I56319_noneck_axial.jpg
51
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIE

95
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_024_S_1171_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070904193900978_S34649_I71473_noneck_coronal.jpg
96
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_018_S_0633_MR_MPR-R____N3__Scaled_Br_20070101222816515_S15573_I35036_noneck_coronal.jpg
97
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_041_S_1368_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070806164945601_S27512_I65248_noneck_axial.jpg
98
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_037_S_0627_MR_MPR__GradWarp__N3__Scaled_Br_20080224135612066_S26830_I92210_noneck_coronal.jpg
99
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_123_S_0162_MR_MPR____N3__Scaled_Br_20070809215316467_S21911_I66697_noneck_coronal.jpg
100
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI

152
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_073_S_0565_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070213213329963_S15762_I39919_noneck_axial.jpg
153
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_023_S_1262_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071002183802518_S38817_I76649_noneck_axial.jpg
154
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_073_S_0565_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070927080020409_S38855_I75476_noneck_axial.jpg
155
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_024_S_1307_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070731173131780_S27061_I63415_noneck_coronal.jpg
156
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_018_S_0286_MR_MPR____N3__Scaled_2_Br_20081001135226421_S22614_I118783_noneck_coronal.jpg
157
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI

196
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_027_S_1385_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071229183142945_S39646_I86386_noneck_coronal.jpg
197
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_014_S_0328_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070828104949893_S29914_I70661_noneck_axial.jpg
198
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_027_S_0404_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061222194841960_S13866_I34204_noneck_coronal.jpg
199
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_023_S_1262_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001160410209_S26314_I118923_noneck_coronal.jpg
200
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_005_S_1341_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070717180348670_S27673_I60417_noneck_coronal.jpg
201
D:\ADNI - IDA

...

816
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_006_S_0547_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070727172259762_S27503_I62561_noneck_coronal.jpg
817
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_011_S_0183_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080114104012072_S29327_I87324_noneck_coronal.jpg
818
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_116_S_0370_MR_MPR-R__GradWarp__N3__Scaled_Br_20080117114434893_S32284_I87744_noneck_axial.jpg
819
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_032_S_0147_MR_MPR____N3__Scaled_2_Br_20081003100226549_S11186_I119079_noneck_coronal.jpg
820
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_021_S_0642_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816134707102_S35279_I68067_noneck_axial.jpg
821
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRE

863
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_007_S_0316_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001120625745_S12583_I118693_noneck_axial.jpg
864
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_036_S_0760_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070209144410346_S18264_I38652_noneck_coronal.jpg
865
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_114_S_0979_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070816152012702_S32809_I68163_noneck_coronal.jpg
866
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_033_S_1281_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080410114734088_S47026_I102012_noneck_axial.jpg
867
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_018_S_0286_MR_MPR____N3__Scaled_Br_20070812130040381_S32252_I67104_noneck_coronal.jpg
868
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T

918
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_032_S_1101_MR_MPR____N3__Scaled_2_Br_20081003104506735_S22783_I119116_noneck_coronal.jpg
919
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_002_S_0938_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071110105158515_S41834_I81312_noneck_axial.jpg
920
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_002_S_0938_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071110105158515_S41834_I81312_noneck_coronal.jpg
921
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_036_S_0760_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070731180424579_S29518_I63445_noneck_axial.jpg
922
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_029_S_0999_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071027175334911_S39410_I78912_noneck_coronal.jpg
923
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\AD

968
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_020_S_0213_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070718120657794_S22879_I60609_noneck_axial.jpg
969
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_012_S_0689_MR_MPR____N3__Scaled_Br_20070923132424980_S36501_I74656_noneck_axial.jpg
970
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_062_S_0690_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070913162253223_S36705_I73145_noneck_axial.jpg
971
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_018_S_0682_MR_MPR____N3__Scaled_Br_20070808165106608_S25847_I65971_noneck_axial.jpg
972
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_027_S_1082_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070327161431010_S22938_I47204_noneck_axial.jpg
973
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMI

1016
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_006_S_0547_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070926110116880_S19779_I75279_noneck_axial.jpg
1017
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_031_S_0321_MR_MPR__GradWarp__N3__Scaled_Br_20070807093706127_S12365_I65383_noneck_axial.jpg
1018
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_023_S_0916_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080114095645209_S30418_I87302_noneck_axial.jpg
1019
D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\ADNI_027_S_1081_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20071028195713680_S38366_I79200_noneck_axial.jpg
fin de imagenes generadas..


In [32]:
#shutil.rmtree(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\AD\GENERADOS\')
#shutil.rmtree(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\VALIDACION\AD\GENERADOS\\')
#shutil.rmtree(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\PRUEBA\AD\GENERADOS\\')
            
           
#shutil.rmtree(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\ENTRENAMIENTO\CN\GENERADOS\\')
#shutil.rmtreer(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\VALIDACION\CN\GENERADOS\\')
#shutil.rmtree(r'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO\PRUEBA\CN\GENERADOS\\')

In [ ]:
# import shutil

# for datos in datos_faltantes:
    
#     try:
#         dir_generados = datos['path_generados']
#         print(dir_generados)      
#         shutil.rmtree(dir_generados)
#     except:    
#         print('Ruta no existe:', dir_generados)        

In [63]:
# validación total de datos generados

for datos in datos_faltantes:
    dir_original = datos['path_original']
    dir_generados = datos['path_generados']
      
    # dir original
    col_dir_or =  dir_original + '/*.jpg'
    col_len_or = len(imread_collection(col_dir_or))
    
    # dir generados
    col_dir_gen =  dir_generados + '/*.jpg'
    col_len_gen = len(imread_collection(col_dir_gen))
    
    print(datos['clase'], '>', datos['tipo_conjunto'])   
    print('_'*50)
    
    print('1.   leidos dir original:', col_len_or)
    print('1.1. datos originales:', datos['datos_actuales'], '\n')
    
    print('2.   leidos dir generados:', col_len_gen)
    print('2.1. datos a generar:', datos['datos_a_generar'], '\n')
    
    print('3.   leidos dir generados y originales:', col_len_or + col_len_gen)
    print('3.1. dir generados y originales:', datos['datos_total'])
    
    print('*'*50)

AD > ENTRENAMIENTO
__________________________________________________
1.   leidos dir original: 762
1.1. datos originales: 762 

2.   leidos dir generados: 1019
2.1. datos a generar: 1019 

3.   leidos dir generados y originales: 1781
3.1. dir generados y originales: 1781
**************************************************
AD > VALIDACION
__________________________________________________
1.   leidos dir original: 181
1.1. datos originales: 181 

2.   leidos dir generados: 240
2.1. datos a generar: 242 

3.   leidos dir generados y originales: 421
3.1. dir generados y originales: 423
**************************************************
AD > PRUEBA
__________________________________________________
1.   leidos dir original: 9
1.1. datos originales: 9 

2.   leidos dir generados: 11
2.1. datos a generar: 13 

3.   leidos dir generados y originales: 20
3.1. dir generados y originales: 22
**************************************************
CN > ENTRENAMIENTO
___________________________________

### Datos totales

In [47]:
# -----------------------------------------------------------
# Clasificacion	     Entrenamiento	    Validacion	  Prueba
#     AD		     762 + 1019	        181 + 242	  9 + 13
#     CN		     1128 + 653	        268	+ 155     14 + 8
#     MCI		     1781	            423	          22
# -----------------------------------------------------------
# Clasificacion	     Entrenamiento	    Validacion	  Prueba
#     AD		     1781	            423   	      22
#     CN		     1781	            423           22
#     MCI		     1781	            423	          22
# -----------------------------------------------------------
# Clasificacion	     5343               1269          66

### Fuentes de datos de acceso abierto

A continuación otra pequeña muestra de fuentes de datos de acceso abierto: 
      
+ [Kaggle](https://www.kaggle.com/datasets). Se creó para ayudar a las empresas a organizar concursos de ciencia de datos para resolver problemas complejos utilizando datos. 
+ [FiveThirtyEight](https://data.fivethirtyeight.com/). Datos de periodismo. Datos detras de las noticias de política, ciencia, cultura, deportes, entre otros. Principalmente contiene información de Estados Unidos.

# References

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>Efe, D. (2021, abril 18). _Jupyter Notebook Tutorial Español #5. Uso de imágenes y fotos con Python, PIL, Numpy y Matplotlib_. [URL](https://www.youtube.com/watch?v=N8F5NVl-ksE)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>M, j. (2020, mayo 14 ). _Contraste y reducción de ruido en imágenes_. [URL](hhttps://www.youtube.com/watch?v=UhW8Kxsnu2g)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>M, u. (2021, julio 1). _Procesamiento de imágenes con Open CV - Google Colab_. [URL](https://www.youtube.com/watch?v=Ai3VmTWeXXk)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>AlgoToGo. (2020, junio 21). _Brain Tumor Detection Using CNN with Python Tensorflow Sklearn OpenCV Part1 Data Processing with CV2_. [URL](https://www.youtube.com/watch?v=bSyY8_rTxfs)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>OMES. (2019, abril 23). _CONTORNOS y como DIBUJARLOS en OpenCV y Python_. [URL](https://www.youtube.com/watch?v=YyRIvbLC99U)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>sMVP.es. (2022, marzo 16). _Python: quitar el fondo de una imagen con 5 líneas de código_. [URL](https://www.youtube.com/watch?v=4HiESVKt700)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>Quintanar, L. F. (2020, noviembre 17). _OpenCV + Python: Localización de contornos para conteo de objetos_. [URL](https://www.youtube.com/watch?v=TK2ICxT4Up8)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>Pacheco, C. O. (2020, octubre 7). _Segmentación de imagen con OpenCV_. [URL](https://www.youtube.com/watch?v=D70vxRJJr_c)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>Datapeaker. (2021, septiembre). _Extracción de craneosegmentación de imágenes en imágenes de resonancia magnética 3d_. [URL](https://datapeaker.com/big-data/extraccion-de-craneo-segmentacion-de-imagenes-en-imagenes-de-resonancia-magnetica-3d/)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>unioviedo.es (2015). _Introducción al procesamiento digital de imágenes con Python_. [URL](https://www.unioviedo.es/compnum/laboratorios_py/Intro_imagen/introduccion_imagen.html)

<a name="cite-PER-GRA:2007"/><sup>[^](#ref-2) </sup>Huber, N. R., Missert, A. D., & Erickson, B. J. (2020, septiembre 30). _Magician’s Corner: 7. Using Convolutional Neural Networks to Reduce Noise in Medical Images_. [URL](https://pubs.rsna.org/doi/full/10.1148/ryai.2020200036)

